<a href="https://colab.research.google.com/github/Arjun-krish-10/NLP/blob/main/Hugging_face_NLP_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install transformers

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

#Zero shot learning

In [ ]:
from transformers import pipeline


In [ ]:
classifier = pipeline("zero-shot-classification",
                      model="valhalla/distilbart-mnli-12-9")

In [ ]:
sequence_to_classify = "Roger Fedderer won the World championship"
candidate_labels = ['Football','Cricket','Tennis','Chess']
hypothesis_template = 'The sport being talked is {}'
classifier(sequence_to_classify, candidate_labels,hypothesis_template=hypothesis_template)

{'labels': ['Tennis', 'Chess', 'Football', 'Cricket'],
 'scores': [0.5370711088180542,
  0.28500232100486755,
  0.10809198021888733,
  0.06983468681573868],
 'sequence': 'Roger Fedderer won the World championship'}

In [ ]:
# !pip install transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import  TFAutoModelForQuestionAnswering
import tensorflow as tf
from transformers import  TFAutoModelForSequenceClassification
from transformers import TFAutoModelWithLMHead

In [ ]:

tokenizer_bart_large = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
model_zero_shot = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')


In [ ]:
# pose sequence as a NLI premise and label (politics) as a hypothesis
# https://joeddav.github.io/blog/2020/05/29/ZSL.html
premise = 'Sachin Tendulkar is playing the world championship'
hypothesis = 'This text is about chess.'

# run through model pre-trained on MNLI
input_ids = tokenizer_bart_large.encode(premise, hypothesis, return_tensors='pt')
logits = model_zero_shot(input_ids)[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,1,2]]
probs = entail_contradiction_logits.softmax(dim=1)
true_prob = probs[:,1].item() * 100
print(f'Probability that the label is true: {true_prob:0.2f}%')

Probability that the label is true: 0.17%


#Question & Answering

In [ ]:


tokenizer_qa = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
model_qa = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")



All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-uncased-distilled-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [ ]:
text = r"""
Ip Man is a Kung Fu movie about the legendary martial arts teacher of the same name. It is rated a staggering 8 out of 10 on IMDb and considered a cult classic among fans.

The movie is almost two hours long, but if you skim through it, you’ll notice: There’s not a lot of fighting. Isn’t that what Kung Fu movies are about? Apparently not.

You’ll see the master having tea, helping his friends, and struggling with everyday life. You’ll see him muse about politics, about war, and about philosophy. You’ll even see Ip Man spending time with his family and training with “ the wooden man ,” a tool he invented.

Why do people love this movie so much if, as it turns out, there are only three major fight scenes? They love it because each fight means something . That’s what Kung Fu is really about: Learning to use martial arts when it matters .

Ip Man perfectly represents this ideal, and that’s why he’s an admirable character. He doesn’t fight just to fight. He wants to maintain peace among his community. Only if the fighting serves a higher goal does he break out his fists.

In the first fight, Ip Man must defend his home against an intruder. In the second, he avenges a friend to send a message. In the third, he makes an example of the leader of the Japanese, occupying forces.

Family, loyalty, and culture. Those are the themes behind Ip Man’s fights, and they’re much bigger than just himself. That’s why it’s an honor to watch him fight and easy to root for him when he does.

There are a lot of lessons in the movie about values, about civility, and about the true philosophy of Kung Fu, but the main one may be: “Don’t fight when it doesn’t matter.” Focus your energy on the biggest obstacles so you may overcome them when they appear.
"""

questions = [
"What is the main lesson in the article?"
# "What does 🤗 Transformers provide?",
# "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
  inputs = tokenizer_qa(question, text, add_special_tokens=True, return_tensors="tf")
  input_ids = inputs["input_ids"].numpy()[0]

  outputs = model_qa(inputs)
  # print(outputs)
  answer_start_scores = outputs.start_logits
  answer_end_scores = outputs.end_logits
  # print(answer_start_scores,answer_end_scores)
  answer_start = tf.argmax(
      answer_start_scores, axis=1
  ).numpy()[0]  # Get the most likely beginning of answer with the argmax of the score
  
  answer_end = (
      tf.argmax(answer_end_scores, axis=1) + 1
  ).numpy()[0]  # Get the most likely end of answer with the argmax of the score
  print(answer_start,answer_end)

  answer = tokenizer_qa.convert_tokens_to_string(tokenizer_qa.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

  print(f"Question: {question}")
  print(f"Answer: {answer}")

390 400
Question: What is the main lesson in the article?
Answer: don ’ t fight when it doesn ’ t matter


In [ ]:
import numpy as np

#Classification

In [ ]:
  
tokenizer_clf = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")

model_clf = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model = TFRobertaForSequenceClassification.from_pretrained("roberta-large-mnli")

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = 'It was a good movie'
tok = tokenizer_clf.encode_plus(text,add_special_tokens=True,return_tensors='tf')

In [ ]:
# ip = np.array(tok['input_ids'])

In [ ]:
output = model_clf(tok)

In [ ]:
output.logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-4.2403626,  4.6366396]], dtype=float32)>

In [ ]:
tf.nn.softmax(
    output.logits, axis=1, name=None
)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1.3954236e-04, 9.9986041e-01]], dtype=float32)>

#Masked identification

In [ ]:

# import tensorflow as tf
tokenizer_mask = AutoTokenizer.from_pretrained("distilbert-base-cased")
model_mask = TFAutoModelWithLMHead.from_pretrained("distilbert-base-cased")


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_tf_auto.py:533: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model

In [ ]:
sequence = f"The book was a romantic {tokenizer_mask.mask_token} genre"
input = tokenizer_mask.encode(sequence, return_tensors="tf")
mask_token_index = tf.where(input == tokenizer.mask_token_id)[0, 1]
token_logits = model_mask(input)[0]
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = tf.math.top_k(mask_token_logits, 5).indices.numpy()

In [ ]:
for token in top_5_tokens:
     print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

The book was a romantic fantasy genre
The book was a romantic comedy genre
The book was a romantic romance genre
The book was a romantic fiction genre
The book was a romantic novel genre
